In [179]:
# Import important library

import pandas as pd
import numpy as np

import matplotlib.pyplot as mplot
import seaborn as sns

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Q1 Read the input file and check the data dimension

In [153]:
#Read input file and understand the data
# "default" is my dependent variable

In [180]:
Gcredit = pd.read_csv("german_credit.csv")

Gcredit.head(5)
#Gcredit = Gcredit.dropna(axis = 0, how ='any') 

,default,account_check_status,duration_in_month,credit_history,purpose,credit_amount,savings,present_emp_since,installment_as_income_perc,personal_status_sex,...,present_res_since,property,age,other_installment_plans,housing,credits_this_bank,job,people_under_maintenance,telephone,foreign_worker
0,0,< 0 DM,6,critical account/ other credits existing (not ...,domestic appliances,1169,unknown/ no savings account,.. >= 7 years,4,male : single,...,4,real estate,67,none,own,2,skilled employee / official,1,"yes, registered under the customers name",yes
1,1,0 <= ... < 200 DM,48,existing credits paid back duly till now,domestic appliances,5951,... < 100 DM,1 <= ... < 4 years,2,female : divorced/separated/married,...,2,real estate,22,none,own,1,skilled employee / official,1,none,yes
2,0,no checking account,12,critical account/ other credits existing (not ...,(vacation - does not exist?),2096,... < 100 DM,4 <= ... < 7 years,2,male : single,...,3,real estate,49,none,own,1,unskilled - resident,2,none,yes
3,0,< 0 DM,42,existing credits paid back duly till now,radio/television,7882,... < 100 DM,4 <= ... < 7 years,2,male : single,...,4,if not A121 : building society savings agreeme...,45,none,for free,1,skilled employee / official,2,none,yes
4,1,< 0 DM,24,delay in paying off in the past,car (new),4870,... < 100 DM,1 <= ... < 4 years,3,male : single,...,4,unknown / no property,53,none,for free,2,skilled employee / official,2,none,yes


In [181]:
# Lets build a Ensemble model but need to modify the dataset first

print('The distrubution of the target is')
print(Gcredit['default'].value_counts())

print("The total NOTNULL values in columns is " , sum([True for idx,row in Gcredit.iterrows() if any(row.isnull())]))

The distrubution of the target is
0    700
1    300
Name: default, dtype: int64
The total NOTNULL values in columns is  0


In [182]:
### Q2.Prepare the model data by converting non-numeric to dummy ( 1 Marks)
##### Hint: Use get_dummies

import os

Gcredit_dummies = pd.get_dummies(data=Gcredit) #, columns= ['default']) 

# Check for File existence before file creation.
if not os.path.isfile('dummies.csv'):
   Gcredit_dummies.to_csv('dummies.csv', sep='\t', encoding='utf-8')
else: # IF exists then delete the file and create new one.
   os.remove('dummies.csv') 
   Gcredit_dummies.to_csv('dummies.csv', sep='\t', encoding='utf-8')

print('New CSV file created')
#print(Gcredit_dummies.head(5))

New CSV file created


In [183]:
# Print Shape of model data
print(Gcredit.shape)
print(Gcredit_dummies.shape)

(1000, 21)
(1000, 62)


### Check for highly correlated variables but don't required any treatment for this use case

In [184]:
Gcredit.corr()
Gcredit_dummies.corr()

,default,duration_in_month,credit_amount,installment_as_income_perc,present_res_since,age,credits_this_bank,people_under_maintenance,account_check_status_0 <= ... < 200 DM,account_check_status_< 0 DM,...,housing_own,housing_rent,job_management/ self-employed/ highly qualified employee/ officer,job_skilled employee / official,job_unemployed/ unskilled - non-resident,job_unskilled - resident,telephone_none,"telephone_yes, registered under the customers name",foreign_worker_no,foreign_worker_yes
default,1.000000,0.214927,0.154739,0.072404,0.002967,-0.091127,-0.045732,-0.003015,0.119581,0.258333,...,-0.134589,0.092785,0.040559,-0.013559,0.005951,-0.021822,0.036466,-0.036466,-0.082079,0.082079
duration_in_month,0.214927,1.000000,0.624984,0.074749,0.034067,-0.036136,-0.011284,-0.023834,0.089452,0.022244,...,-0.075169,-0.064417,0.147515,0.055010,-0.044043,-0.181203,-0.164718,0.164718,-0.138196,0.138196
credit_amount,0.154739,0.624984,1.000000,-0.271316,0.028926,0.032716,0.020795,0.017142,0.119612,-0.020912,...,-0.117497,-0.024611,0.319715,-0.092636,-0.027969,-0.161757,-0.276995,0.276995,-0.050050,0.050050
installment_as_income_perc,0.072404,0.074749,-0.271316,1.000000,0.049302,0.058266,0.021669,-0.071207,-0.051906,0.046917,...,0.049922,-0.091373,0.042805,0.042623,-0.087834,-0.057237,-0.014413,0.014413,-0.090024,0.090024
present_res_since,0.002967,0.034067,0.028926,0.049302,1.000000,0.266419,0.089625,0.042643,-0.055817,0.088350,...,-0.297547,0.167285,0.004952,-0.000657,-0.034545,0.009065,-0.095359,0.095359,-0.054097,0.054097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
job_unskilled - resident,-0.021822,-0.181203,-0.161757,-0.057237,0.009065,0.043712,-0.010392,0.145066,0.018041,0.023542,...,0.063003,0.014347,-0.208392,-0.652438,-0.074991,1.000000,0.253720,-0.253720,0.087412,-0.087412
telephone_none,0.036466,-0.164718,-0.276995,-0.014413,-0.095359,-0.145259,-0.065553,0.014753,-0.015276,0.067149,...,0.036275,0.049524,-0.385706,0.061289,0.040124,0.253720,1.000000,-1.000000,0.107401,-0.107401
"telephone_yes, registered under the customers name",-0.036466,0.164718,0.276995,0.014413,0.095359,0.145259,0.065553,-0.014753,0.015276,-0.067149,...,-0.036275,-0.049524,0.385706,-0.061289,-0.040124,-0.253720,-1.000000,1.000000,-0.107401,0.107401
foreign_worker_no,-0.082079,-0.138196,-0.050050,-0.090024,-0.054097,-0.006151,-0.009717,0.077071,-0.059172,0.057751,...,0.018960,0.032849,-0.051858,-0.047292,0.042834,0.087412,0.107401,-0.107401,1.000000,-1.000000


In [185]:
Gcredit_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 62 columns):
default                                                                       1000 non-null int64
duration_in_month                                                             1000 non-null int64
credit_amount                                                                 1000 non-null int64
installment_as_income_perc                                                    1000 non-null int64
present_res_since                                                             1000 non-null int64
age                                                                           1000 non-null int64
credits_this_bank                                                             1000 non-null int64
people_under_maintenance                                                      1000 non-null int64
account_check_status_0 <= ... < 200 DM                                        1000 non-null uint8
account_chec

### Drop the original variables which are converted to dummy

In [186]:
print("OBSERVATION")
print("The dummies dataframe does not have the original variables which are categorical, which are converted.")
#Gcredit_dummies.drop("account_check_status",axis=1)
#Gcredit_dummies.drop("credit_history",axis=1)
#Gcredit_dummies.drop("purpose",axis=1)
#Gcredit_dummies.drop("credit_amount",axis=1)
#Gcredit_dummies.drop("savings",axis=1)
#Gcredit_dummies.drop("present_emp_since",axis=1)
#Gcredit_dummies.drop("personal_status_sex",axis=1)
#Gcredit_dummies.drop("other_debtors",axis=1)
#Gcredit_dummies.drop("property",axis=1)
#Gcredit_dummies.drop("other_installment_plans",axis=1)
#Gcredit_dummies.drop("housing",axis=1)
#Gcredit_dummies.drop("job",axis=1)
#Gcredit_dummies.drop("housing",axis=1)
#Gcredit_dummies.drop("telephone",axis=1)
#Gcredit_dummies.drop("foreign_worker",axis=1)

print("The dummies dataframe do have the original variables which are numerical, which are dropped as asked.")

Gcredit_dummies.drop("duration_in_month",axis=1)
Gcredit_dummies.drop("credit_amount",axis=1)
Gcredit_dummies.drop("installment_as_income_perc",axis=1)
Gcredit_dummies.drop("present_res_since",axis=1)
Gcredit_dummies.drop("age",axis=1)
Gcredit_dummies.drop("credits_this_bank",axis=1)
Gcredit_dummies.drop("people_under_maintenance",axis=1)


OBSERVATION
The dummies dataframe does not have the original variables which are categorical, which are converted.
The dummies dataframe do have the original variables which are numerical, which are dropped as asked.


,default,duration_in_month,credit_amount,installment_as_income_perc,present_res_since,age,credits_this_bank,account_check_status_0 <= ... < 200 DM,account_check_status_< 0 DM,account_check_status_>= 200 DM / salary assignments for at least 1 year,...,housing_own,housing_rent,job_management/ self-employed/ highly qualified employee/ officer,job_skilled employee / official,job_unemployed/ unskilled - non-resident,job_unskilled - resident,telephone_none,"telephone_yes, registered under the customers name",foreign_worker_no,foreign_worker_yes
0,0,6,1169,4,4,67,2,0,1,0,...,1,0,0,1,0,0,0,1,0,1
1,1,48,5951,2,2,22,1,1,0,0,...,1,0,0,1,0,0,1,0,0,1
2,0,12,2096,2,3,49,1,0,0,0,...,1,0,0,0,0,1,1,0,0,1
3,0,42,7882,2,4,45,1,0,1,0,...,0,0,0,1,0,0,1,0,0,1
4,1,24,4870,3,4,53,2,0,1,0,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,12,1736,3,4,31,1,0,0,0,...,1,0,0,0,0,1,1,0,0,1
996,0,30,3857,4,4,40,1,0,1,0,...,1,0,1,0,0,0,0,1,0,1
997,0,12,804,4,4,38,1,0,0,0,...,1,0,0,1,0,0,1,0,0,1
998,1,45,1845,4,4,23,1,0,1,0,...,0,0,0,1,0,0,0,1,0,1


### Q3 Split Train/Test data 70:30 ratio( 1 Marks)
##### Hint:from sklearn.model_selection import train_test_split

In [187]:
y=Gcredit_dummies['default']
X=Gcredit_dummies.drop('default',axis=1)

In [188]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=0)

### Q4 Build Random Forest Model( 1 Marks)
#### Hint:from sklearn.ensemble import RandomForestClassifier using n_jobs=2,n_estimators=500,criterion="entropy",random_state=9999

In [189]:
#clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)

RFC = RandomForestClassifier(n_jobs=2,n_estimators=500,criterion="entropy",random_state=9999)
RFC.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=2,
            oob_score=False, random_state=9999, verbose=0,
            warm_start=False)

### Q5 Calculate Confusion Matrix and Accuracy score (1 Marks)
##### Hint: Use confusion_matrix and accuracy_score

In [190]:
y_predict = RFC.predict(X)
RFC_score = RFC.score(X_test, y_test)
print(RFC_score)

0.76


In [191]:
print(y_test.shape)
print(y_predict.shape)

print(metrics.confusion_matrix(y, y_predict))

(300,)
(1000,)
[[676  24]
 [ 48 252]]


### Q6 Show the list of the features importance( 1 Marks)

In [167]:
print(RFC.feature_importances_)

[0.10931092 0.03821793 0.03914787 0.08402943 0.02076322 0.01375277
 0.01775856 0.03748315 0.00943311 0.04933653 0.01231516 0.02058598
 0.01113965 0.01478863 0.01355963 0.00710426 0.01112178 0.01703273
 0.01380554 0.01486529 0.00528099 0.00227354 0.013509   0.00322552
 0.00140134 0.00562942 0.01801239 0.0087405  0.00795285 0.01533444
 0.01443549 0.01698777 0.01651539 0.0135758  0.00685997 0.01647879
 0.00661597 0.00944195 0.01665183 0.00759396 0.010149   0.009597
 0.01405776 0.01549888 0.02041232 0.01113765 0.01369806 0.0164813
 0.00678649 0.00686581 0.01499368 0.01272549 0.0118058  0.01726226
 0.00295069 0.0135274  0.01598347 0.01571637 0.00419553 0.00408597]


### Q7 K-fold cross-validation( 2 Marks)
##### k-fold cross validation( without stratification)
##### Usually k is set as 10-20 in practical settings, depends on data set size

In [168]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [169]:
# Use below values
num_folds = 10
seed = 77

In [101]:
#Validate the Random Forest model build above using k fold

In [170]:
# Bagged Decision Trees for Classification
# from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
#seed = 7

#kfold = model_selection.KFold(n_splits=10, random_state=seed)
#cart = DecisionTreeClassifier()
#num_trees = 100
#model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)

kfold = model_selection.KFold( n_splits=10, random_state=seed)
results = model_selection.cross_val_score(model, X, y, cv=kfold)
print(results)

[0.86 0.66 0.8  0.76 0.73 0.69 0.73 0.73 0.72 0.7 ]


In [103]:
#Calculate Mean score

In [171]:
print(results.mean())

0.738


In [ ]:
# Calculate score standard deviation using std()

In [172]:
results.std()

0.054369108876272755

# Q8 Print the confusion matrix( 1 Marks)

In [173]:
print(metrics.confusion_matrix(y, y_predict))

[[679  21]
 [ 48 252]]


# Q9.Classification accuracy: 
percentage of correct predictions and Calculate sensitivity (or True Positive Rate or Recall) and Precision.
(2 Marks)

In [174]:
# summarize the fit of the model
print(metrics.classification_report(y, y_predict))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       700
           1       0.92      0.84      0.88       300

   micro avg       0.93      0.93      0.93      1000
   macro avg       0.93      0.91      0.92      1000
weighted avg       0.93      0.93      0.93      1000



[link text](https://)### Bootstrapping ( Bonus and Optional)
##### Given a dataset of size n, a bootstrap sample is created by sampling n instances uniformly from the data (with/without replacement)
##### Create a model with each bootstrap sample and validate it with the test set
##### Final result is calculated by averaging the accuracy of models

In [175]:
# Number of iterations for bootstrapping
bootstrap_iteration = 10
accuracy = []

In [176]:
from sklearn.utils import resample
from sklearn.metrics import accuracy_score

for i in range(bootstrap_iteration):
    X_, y_ = resample(X_train, y_train)
    RFC.fit(X_, y_)
    y_pred = RFC.predict(X_test)
    acc = accuracy_score(y_pred, y_test)
    accuracy.append(acc)

In [177]:
accuracy = np.array(accuracy)
print('Accuracy Score')
print('Average: ', accuracy.mean())
print('Standard deviation: ', accuracy.std())

Accuracy Score
Average:  0.7656666666666666
Standard deviation:  0.010005554013202429
